<a href="https://colab.research.google.com/github/agyanigiri/DLFS/blob/main/DLFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np

# Operations

In [2]:
class Operation:
  def __init__(self):
    self.input_ = None
  
  def forward(self, input):
    self.input_ = input
    self.output = self._output()
    return self.output

  def backward(self, output_grad):
    self.input_grad = self._input_grad(output_grad)
    return self.input_grad

  def _output(self):
    raise NotImplementedError()

  def _input_grad(self, output_grad):
    raise NotImplementedError()

In [3]:
class ParamOperation(Operation):
  def __init__(self, param):
    super().__init__()
    self.param = param

  def backward(self, output_grad):
    self.input_grad = self._input_grad(output_grad)
    self.param_grad = self._param_grad(output_grad)
    return self.input_grad

  def _param_grad(self, output_grad):
    raise NotImplementedError()

In [4]:
class WeightMultiply(ParamOperation):
  def __init__(self, param):
    super().__init__(param)

  def _output(self):
    return self.input_ @ self.param

  def _input_grad(self, output_grad):
    return output_grad @ self.param.T

  def _param_grad(self, output_grad):
    return self.input_.T @ output_grad

In [7]:
class BiasAdd(ParamOperation):
  def __init__(self, param):
    super().__init__(param)
  
  def _output(self):
    return self.input_ + self.param

  def _input_grad(self, output_grad):
    return output_grad

  def _param_grad(self, output_grad):
    return output_grad.sum(axis=0, keepdims=True)

In [8]:
class Linear(Operation):
  def __init__(self):
    super().__init__()
  
  def _output(self):
    return self.input_

  def _input_grad(self, output_grad):
    return output_grad

In [9]:
class Sigmoid(Operation):
  def __init__(self):
    super().__init__()

  def _output(self):
    return 1.0 / (1 + np.exp(-1.0 * self.input_))

  def _input_grad(self):
    return self.output * (1.0 - self.output)

# Layers

# Optimizers